## Transfer learning

In [ ]:
data_dir = './dataset/trigo/enfermo/'
data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg'))) + len(list(data_dir.glob('*/*.png')))
image_count

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != ".ipynb_checkpoints"])
CLASS_NAMES

# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  validation_split=0.15,                            
                                                                  zoom_range=0.4,
                                                                  horizontal_flip=True,
                                                                  rotation_range=90
                                                                 )

BATCH_SIZE = 16
# IMG_HEIGHT = 224
# IMG_WIDTH = 224
IMG_HEIGHT = 448
IMG_WIDTH = 448
STEPS_PER_EPOCH = np.ceil((image_count*0.85)/BATCH_SIZE)
int(STEPS_PER_EPOCH)

train_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "training",
                                                     # save_to_dir = '/tf/data/Tesis/dataset/augmented',
                                                     # save_prefix = 'trigo_',
                                                     classes = list(CLASS_NAMES))

valid_data_gen = image_generator.flow_from_directory(directory= str(data_dir),
                                                     batch_size= BATCH_SIZE,
                                                     shuffle= True,
                                                     target_size= (IMG_HEIGHT, IMG_WIDTH),
                                                     subset = "validation",
                                                     classes = list(CLASS_NAMES))

model = tf.keras.models.load_model('models/plant_village_trigo_resnet_448x448.11-0.65.h5')

predicts = model.predict_generator(valid_data_gen)

array_pred = []
for i in range(len(predicts)):
    array_pred.append(np.argmax(predicts[i]))

tags_names = valid_data_gen.filenames

def index_value(clas_file):
    clas_file = clas_file.split('/')[0]
    return list(CLASS_NAMES).index(clas_file)

index_class = map(index_value, tags_names)

index_class = list(index_class)

In [ ]:
!pip install sklearn

from sklearn.metrics import confusion_matrix

confusion =confusion_matrix(index_class,array_pred)

confusion